In [153]:
import os
import cv2
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
import numpy as np

In [154]:
#extra block 4 testing

import os
import cv2

images = []
labels = []
folder = '/home/tom/apps/python-envs/data/stop'

for filename in os.listdir(folder):
    #print(filename)
    img_path = os.path.join(folder, filename)
    
    # Attempt to load the image
    img = cv2.imread(img_path)
    
    # Check if the image is loaded successfully
    if img is not None:
        img = cv2.resize(img, (224, 224))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        images.append(img)
        
        # Use 'stop' in img_path instead of 'stop' in folder
        labels.append(1 if 'stop' in img_path else 0)
    else:
        print(f"Error loading image: {img_path}")

# Now, 'images' and 'labels' should contain your processed data

In [155]:
# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, padding=2), # input = 3 meaning R, G and B
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(128 * 14 * 14, 2) # 14 here because 224 (input dimension of image) / 2 * 4 (pooling size * number of layers), there is 2 output (0 for not_stop and 1 for stop)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out



In [156]:
# Function to load images from a folder
def load_images(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        img = cv2.resize(img, (224, 224))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        images.append(img)
        labels.append(1 if folder == '/home/tom/apps/python-envs/data/stop' else 0)  # 1 for stop, 0 for not stop
    return np.array(images), np.array(labels)

In [157]:
# Load images and labels
stop_images, stop_labels = load_images('/home/tom/apps/python-envs/data/stop')
not_stop_images, not_stop_labels = load_images('/home/tom/apps/python-envs/data/not_stop')

# Combine stop and not_stop images
images = np.concatenate((stop_images, not_stop_images), axis=0)
labels = np.concatenate((stop_labels, not_stop_labels), axis=0)

# Define transformations and create a DataLoader
transform = transforms.Compose([transforms.ToTensor()])


# Convert images to PyTorch tensors and normalize
image_tensors = torch.tensor(images.transpose((0, 3, 1, 2)), dtype=torch.float32) / 255.0
label_tensors = torch.tensor(labels, dtype=torch.long)

# Create a TensorDataset
dataset = torch.utils.data.TensorDataset(image_tensors, label_tensors)


dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [158]:
# Instantiate the model
model = SimpleCNN()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss() # is it better to use bce loss for binary classification?
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10  # You can adjust this based on your dataset size and convergence
for epoch in range(num_epochs):
    for images_batch, labels_batch in dataloader:
        optimizer.zero_grad()
        outputs = model(images_batch)
        loss = criterion(outputs, labels_batch)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save the trained model
torch.save(model.state_dict(), 'trained_model.pth')

Epoch [1/10], Loss: 0.6888
Epoch [2/10], Loss: 0.6861
Epoch [3/10], Loss: 0.5488
Epoch [4/10], Loss: 2.2374
Epoch [5/10], Loss: 0.5674
Epoch [6/10], Loss: 0.4837
Epoch [7/10], Loss: 0.4307
Epoch [8/10], Loss: 0.3664
Epoch [9/10], Loss: 0.3719
Epoch [10/10], Loss: 0.0396


In [159]:
# Function to predict and classify an image
def classify_image(image_path):
    # Read the image using OpenCV
    img = cv2.imread(image_path)
    
    # Resize the image to match the input size of the model
    img = cv2.resize(img, (224, 224))
    
    # Convert the image to RGB color format
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Convert the image to a PyTorch tensor and normalize
    img_tensor = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32) / 255.0
    
    # Add an extra dimension to represent the batch (single image in this case)
    img_tensor = img_tensor.unsqueeze(0)

    # Disable gradient calculation during inference
    with torch.no_grad():
        # Set the model to evaluation mode
        model.eval()
        
        # Perform forward pass to get the output predictions
        output = model(img_tensor)
        
        # Get the predicted class index
        _, predicted = torch.max(output, 1)
        
        # Return the predicted class index as an integer
        return predicted.item()

In [161]:

# Define the base directory for images
base_image_dir = '/home/tom/apps/python-envs/data/'

# Test the classifier on an additional image
test_image_prediction = classify_image(os.path.join(base_image_dir, '4.png'))

print("Test Image Prediction:", test_image_prediction)


Test Image Prediction: 1


In [ ]:
#no need of this block (use if you want percentage)


# Define the base directory for images
base_image_dir = '/home/tom/apps/python-envs/data/'
base_image_dir_stop = '/home/tom/apps/python-envs/data/stop'
base_image_dir_not_stop = '/home/tom/apps/python-envs/data/not_stop'

# Test the classifier on an additional image
test_image_prediction = classify_image(os.path.join(base_image_dir, '4.png'))
print("Test Image Prediction:", test_image_prediction)

# Get a list of all image filenames in the 'stop' directory
stop_image_filenames = [filename for filename in os.listdir(base_image_dir_stop) if filename.endswith('.jpg')]

# Actual labels for 'stop' images
actual_labels_stop = [1] * len(stop_image_filenames)

# Classify each 'stop' image and store the predictions
stop_images_pred = []

for filename in stop_image_filenames:
    image_path = os.path.join(base_image_dir_stop, filename)
    prediction = classify_image(image_path)
    stop_images_pred.append(prediction)

# Get a list of all image filenames in the 'not_stop' directory
not_stop_image_filenames = [filename for filename in os.listdir(base_image_dir_not_stop) if filename.endswith('.jpg')]

# Actual labels for 'not_stop' images
actual_labels_not_stop = [0] * len(not_stop_image_filenames)

# Classify each 'not_stop' image and store the predictions
not_stop_images_pred = []

for filename in not_stop_image_filenames:
    image_path = os.path.join(base_image_dir_not_stop, filename)
    prediction = classify_image(image_path)
    not_stop_images_pred.append(prediction)

# Calculate accuracy for 'stop' images
correct_predictions_stop = sum([1 for pred, actual in zip(stop_images_pred, actual_labels_stop) if pred == actual])
total_images_stop = len(stop_image_filenames)
accuracy_stop = correct_predictions_stop / total_images_stop

# Calculate accuracy for 'not_stop' images
correct_predictions_not_stop = sum([1 for pred, actual in zip(not_stop_images_pred, actual_labels_not_stop) if pred == actual])
total_images_not_stop = len(not_stop_image_filenames)
accuracy_not_stop = correct_predictions_not_stop / total_images_not_stop

# Display the predictions and accuracy for 'stop' images
print("Predictions for 'stop' images:")
for filename, prediction in zip(stop_image_filenames, stop_images_pred):
    print(f"Image '{filename}': Prediction {prediction}")

print(f"Accuracy on 'stop' images: {accuracy_stop * 100:.2f}%")

# Display the predictions and accuracy for 'not_stop' images
print("\nPredictions for 'not_stop' images:")
for filename, prediction in zip(not_stop_image_filenames, not_stop_images_pred):
    print(f"Image '{filename}': Prediction {prediction}")

print(f"Accuracy on 'not_stop' images: {accuracy_not_stop * 100:.2f}%")
